In [ ]:
import digitalhub as dh

In [ ]:
project = dh.get_or_create_project("project-dbt-ci")

In [ ]:
project.refresh().to_dict()

In [ ]:
url = "https://gist.githubusercontent.com/kevin336/acbb2271e66c10a5b73aacf82ca82784/raw/e38afe62e088394d61ed30884dd50a6826eee0a8/employees.csv"
di = project.new_dataitem(name="employees-dbt",
                          kind="table",
                          path=url)

In [ ]:
sql = """
WITH tab AS (
    SELECT  *
    FROM    {{ ref('employees') }}
)
SELECT  *
FROM    tab
WHERE   tab."DEPARTMENT_ID" = '50'
"""

In [ ]:
function = project.new_function(name="function-dbt",
                                kind="dbt",
                                code=sql)

In [ ]:
run = function.run("transform",
                   inputs={"employees": di.key},
                   outputs={"output_table": "department-50"})

One can view the state of 'run' from digitalhub Console application or can query it using the digitalhub sdk API call

In [ ]:
run.refresh().status.state

Wait until data transform 'run' is completed.

In [ ]:
proj = dh.get_or_create_project("project-dbt-ci")
di = proj.get_dataitem('department-50')
df = di.as_df()
df.head()

In [ ]:
import os
new_folder = 'src'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

In [ ]:
%%writefile "src/dbt_ci_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(url):
    with pipeline_context() as pc:
        s1_dataset = pc.step(name="dbt", function="function-dbt", action="transform", inputs={"employees":url}, outputs={"output_table": "department-50"})

In [ ]:
workflow = proj.new_workflow(name="pipeline_dbt", kind="kfp", code_src= "src/dbt_ci_pipeline.py", handler = "myhandler")

In [ ]:
di_url = proj.new_dataitem(name="url_data_item",kind="table",path=url)

In [ ]:
workflow_run = workflow.run(parameters={"url": di_url.key})

Wait until workflow pipeline is executed completely. One can view the state of workflow from the console application or using the digitalhub sdk API call. 

In [ ]:
workflow_run.refresh().status.state

Once completed, one can view the scenario results by fetching the list of newly created dataitems using digithub sdk.

In [ ]:
data_items = dh.list_dataitems(project='project-dbt-ci')

In [ ]:
len(data_items)

As the result of scenario 5 data items are created in the core project. One can use the 'Core' application instance to view them in the project console or can use the digitalhub sdk API to fetch them.